# Part 3, Topic 3: DPA on Firmware Implementation of AES

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

## AES Model

In [179]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWHUSKY'
CRYPTO_TARGET = 'TINYAES128C'
VERSION = 'HARDWARE'

In [188]:
if VERSION == 'HARDWARE':
    %run "Lab 3_3 - DPA on Firmware Implementation of AES (HARDWARE).ipynb"
elif VERSION == 'SIMULATED':
    %run "Lab 3_3 - DPA on Firmware Implementation of AES (SIMULATED).ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍
scope.gain.mode                          changed from low                       to high                     
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.clock.adc_rate                     changed from 0.0                       to 29454545.454545453       
scope.io.tio1     

Capturing traces:   0%|          | 0/25000 [00:00<?, ?it/s]

To check that the ASCON implementation is working, the result of the first random nonce is printed

In [217]:
nonce_0 = textin_array[0]
print("Nonce 0 (hex):", nonce_0.hex())

response_0 = response_array[0]
print("Output (40 bytes):", response_0.hex())


Nonce 0 (hex): 48ffefc6b2125c313d39cc31f99e4ecb
Output (40 bytes): 747f03b24663ffb9be319d4b77afc6c470de72b6ceffff3abf319d4bf7afc685fa128c4bf723c675


definition of the function that performs the constant addition layer and the substitution layer for 1 bit of each state (1 bit of IV, 2 bits of nonce, 2 bits of key), and it returns a binary value s[0] ^ s[3].
s[0] ^ s[3] has been chosen because, by looking at the assembly code of the firmware that perform the partial ASCON permutation, s[0] ^= s[4] gets stored inside the register R0, and then the very next value that gets stored in R0, is s[3] ^= s[2].

In [252]:
def simulate_ascon_lsb(iv, n0, n1, k0_b, k1_b, bitpos=0):
   
    iv_b = (iv >> bitpos) & 1
    n0_b = (n0 >> bitpos) & 1
    n1_b = (n1 >> bitpos) & 1
   

    s = [iv_b, k0_b, k1_b, n0_b, n1_b]

    rc_bit = ((0xF0 >> bitpos) & 1) if 0 <= bitpos < 8 else 0
    s[2] ^= rc_bit

    s[0] ^= s[4]
    s[4] ^= s[3]
    s[2] ^= s[1]

    t = [0] * 5
    t[0] = ((~s[0]) & 1) & s[1]
    t[1] = ((~s[1]) & 1) & s[2]
    t[2] = ((~s[2]) & 1) & s[3]
    t[3] = ((~s[3]) & 1) & s[4]
    t[4] = ((~s[4]) & 1) & s[0]
    for i in range(5):
        s[i] ^= t[(i + 1) % 5]

    s[1] ^= s[0]
    s[0] ^= s[4]
    s[3] ^= s[2]
    s[2] = (~s[2]) & 1

    return s[0] ^ s[3]


Now the 4 combination of the 2 key bit gets tested, so for each one of them, for each nonce tried, the function "simulate_ascon_lsb" gets called and the power traces get splitted according to the return value of the function. 
This process gets then iterated for each 5-bit state. meaning:
round 1: s[0][0], s[1][0], ... , s[4][0]
round 2: s[0][1], s[1][1], ... , s[4][1]
round 3: s[0][2], s[1][2], ... , s[4][2]
then the most likely key bits gets printed

In [261]:
def run_dpa_all_bits(trace_array, textin_array):
    traces = np.array(trace_array)
    IV = 0x00001000808c0001
    best_k0_bits = []
    best_k1_bits = []

    for bitpos in range(32):  # bitpos 0..8
        best_peak = -1
        best_key = None

        for k0_guess in [0, 1]:
            for k1_guess in [0, 1]:
                y = []
                for pt in textin_array:
                    n0 = int.from_bytes(pt[0:8], 'little')
                    n1 = int.from_bytes(pt[8:16], 'little')
                    k0 = k0_guess  # in questa versione solo 1 bit guess
                    k1 = k1_guess
                    predicted_bit = simulate_ascon_lsb(IV, n0, n1, k0, k1, bitpos)
                    y.append(predicted_bit)

                y = np.array(y)
                group0 = traces[y == 0]
                group1 = traces[y == 1]
                mean0 = np.mean(group0, axis=0)
                mean1 = np.mean(group1, axis=0)
                dpa_trace = mean1 - mean0

                peak_val = np.max(np.abs(dpa_trace))
                if peak_val > best_peak:
                    best_peak = peak_val
                    best_key = (k0_guess, k1_guess)

            

        print(f"[bitpos={bitpos}] Best key guess: k0={best_key[0]}, k1={best_key[1]}, peak={best_peak:.4f}")
        best_k0_bits.append(str(best_key[0]))
        best_k1_bits.append(str(best_key[1]))
    key0_str = ''.join(reversed(best_k0_bits))  # reversed se vuoi MSB->LSB
    key1_str = ''.join(reversed(best_k1_bits))
    print(f"key0 bits: {key0_str}")
    print(f"key1 bits: {key1_str}")
   


In [262]:
run_dpa_all_bits(trace_array, textin_array)

[bitpos=0] Best key guess: k0=1, k1=0, peak=0.0072
[bitpos=1] Best key guess: k0=0, k1=0, peak=0.0045
[bitpos=2] Best key guess: k0=0, k1=0, peak=0.0036
[bitpos=3] Best key guess: k0=0, k1=0, peak=0.0035
[bitpos=4] Best key guess: k0=0, k1=0, peak=0.0036
[bitpos=5] Best key guess: k0=0, k1=0, peak=0.0038
[bitpos=6] Best key guess: k0=0, k1=0, peak=0.0052
[bitpos=7] Best key guess: k0=0, k1=0, peak=0.0071
[bitpos=8] Best key guess: k0=0, k1=0, peak=0.0041
[bitpos=9] Best key guess: k0=0, k1=0, peak=0.0020
[bitpos=10] Best key guess: k0=0, k1=0, peak=0.0030
[bitpos=11] Best key guess: k0=0, k1=0, peak=0.0025
[bitpos=12] Best key guess: k0=0, k1=0, peak=0.0047
[bitpos=13] Best key guess: k0=0, k1=0, peak=0.0038
[bitpos=14] Best key guess: k0=0, k1=0, peak=0.0038
[bitpos=15] Best key guess: k0=0, k1=0, peak=0.0069
[bitpos=16] Best key guess: k0=0, k1=0, peak=0.0028
[bitpos=17] Best key guess: k0=0, k1=0, peak=0.0053
[bitpos=18] Best key guess: k0=1, k1=0, peak=0.0058
[bitpos=19] Best key g

trying the 32 msb in case those are the ones saved inside r0

In [263]:
def run_dpa_all_bits(trace_array, textin_array):
    traces = np.array(trace_array)
    IV = 0x00001000808c0001
    best_k0_bits = []
    best_k1_bits = []

    for bitpos in range(32, 64):
        
        best_peak = -1
        best_key = None

        for k0_guess in [0, 1]:
            for k1_guess in [0, 1]:
                y = []
                for pt in textin_array:
                    n0 = int.from_bytes(pt[0:8], 'little')
                    n1 = int.from_bytes(pt[8:16], 'little')
                    k0 = k0_guess  
                    k1 = k1_guess
                    predicted_bit = simulate_ascon_lsb(IV, n0, n1, k0, k1, bitpos)
                    y.append(predicted_bit)

                y = np.array(y)
                group0 = traces[y == 0]
                group1 = traces[y == 1]
                mean0 = np.mean(group0, axis=0)
                mean1 = np.mean(group1, axis=0)
                dpa_trace = mean1 - mean0

                peak_val = np.max(np.abs(dpa_trace))
                if peak_val > best_peak:
                    best_peak = peak_val
                    best_key = (k0_guess, k1_guess)

            

        print(f"[bitpos={bitpos}] Best key guess: k0={best_key[0]}, k1={best_key[1]}, peak={best_peak:.4f}")
        best_k0_bits.append(str(best_key[0]))
        best_k1_bits.append(str(best_key[1]))
    key0_str = ''.join(reversed(best_k0_bits))  
    key1_str = ''.join(reversed(best_k1_bits))
    print(f"key0 bits: {key0_str}")
    print(f"key1 bits: {key1_str}")

In [ ]:
run_dpa_all_bits(trace_array, textin_array)

[bitpos=32] Best key guess: k0=0, k1=0, peak=0.0073
[bitpos=33] Best key guess: k0=0, k1=0, peak=0.0062
[bitpos=34] Best key guess: k0=0, k1=0, peak=0.0029
[bitpos=35] Best key guess: k0=0, k1=0, peak=0.0077
[bitpos=36] Best key guess: k0=0, k1=0, peak=0.0047
[bitpos=37] Best key guess: k0=0, k1=0, peak=0.0056
[bitpos=38] Best key guess: k0=0, k1=0, peak=0.0049
[bitpos=39] Best key guess: k0=0, k1=0, peak=0.0057
[bitpos=40] Best key guess: k0=0, k1=0, peak=0.0046
[bitpos=41] Best key guess: k0=0, k1=0, peak=0.0042
[bitpos=42] Best key guess: k0=0, k1=0, peak=0.0072
[bitpos=43] Best key guess: k0=0, k1=0, peak=0.0026
[bitpos=44] Best key guess: k0=1, k1=0, peak=0.0039
[bitpos=45] Best key guess: k0=0, k1=0, peak=0.0051
[bitpos=46] Best key guess: k0=0, k1=0, peak=0.0052
[bitpos=47] Best key guess: k0=0, k1=0, peak=0.0050
[bitpos=48] Best key guess: k0=0, k1=0, peak=0.0042
[bitpos=49] Best key guess: k0=0, k1=0, peak=0.0069
[bitpos=50] Best key guess: k0=0, k1=0, peak=0.0039
[bitpos=51] 